<a href="https://colab.research.google.com/github/lucia1299/DBMS_InvestigativeJournalism/blob/main/Entity_Metadata_Triples(labelled_dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Processing WN_test.tsv with label

In [ ]:
def process_tsv(file_path, max_tokens=1000):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    sentences_words = []
    sentences_tags = []
    current_sentence_words = []
    current_sentence_tags = []
    sentence_enders = ".!?"
    list_sentences=[]
    list_tags=[]

    for line in lines:
        token = line.strip()
        if token:
            word, tag = token.split('\t')
            current_sentence_words.append(word)
            current_sentence_tags.append(tag)

            if word in sentence_enders:
                sentences_words.append(current_sentence_words)
                sentences_tags.append(current_sentence_tags)
                current_sentence_words = []
                current_sentence_tags = []
        else:
            if current_sentence_words:
                sentences_words.append(current_sentence_words)
                sentences_tags.append(current_sentence_tags)
                current_sentence_words = []
                current_sentence_tags = []

    if current_sentence_words:
        sentences_words.append(current_sentence_words)
        sentences_tags.append(current_sentence_tags)

    for sentence in sentences_words:
        list_sentences.append(sentence)
    for tag_list in sentences_tags:
        list_tags.append(tag_list)

    return list_sentences, list_tags

# Usage example
file_path = '/content/drive/MyDrive/Colab Notebooks/TESI/github/WN_test.tsv'
list_sentences, list_tags = process_tsv(file_path)

In [ ]:
print(list_sentences[1])
print(list_tags[1])

['Il', 'presentatore', 'Valerio', 'Merola', 'è', 'entrato', 'nel', 'mirino', 'della', 'giustizia', 'italiana', ',', 'per', 'problemi', 'con', 'il', 'fisco', '.']
['O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
titles = []
title_count = 0

for sentence in list_sentences:
    if sentence[-1] not in ".;?!:":
        titles.append(sentence)
        title_count += 1

print(f"Number of sentences appended to titles: {title_count}")

Number of sentences appended to titles: 290


In [ ]:
def create_dataset_label(list_sentences, list_tags):
    dataset_list = []
    counterId = 0
    titles = []

    for sentence in list_sentences:
        if sentence[-1] not in ".;?!:":
            titles.append(sentence)

    for i, sentence in enumerate(list_sentences):
        if sentence[-1] not in ".;?!:":
            articles_dict = {}
            articles_dict['id'] = counterId
            articles_dict['title'] = sentence
            articles_dict['article'] = []
            articles_dict['tags'] = []

            for j in range(i + 1, len(list_sentences)):
                if list_sentences[j] in titles:
                    break
                articles_dict['article'].append(list_sentences[j])
                articles_dict['tags'].append(list_tags[j])

            dataset_list.append(articles_dict)
            counterId += 1

    return dataset_list

In [ ]:
dataset_list = create_dataset_label(list_sentences, list_tags)
print(type(dataset_list[0]['tags'][1][0]))

<class 'str'>


In [ ]:
print(dataset_list[1])
print(type(dataset_list[1]))
print(type(dataset_list))

{'id': 1, 'title': ['Rivolta', 'del', 'pane', 'nel', 'Magreb', ',', '50', 'morti', 'in', 'Tunisia'], 'article': [['50', 'morti', 'in', 'Tunisia', ',', 'scontri', 'in', 'Algeria', 'e', 'tensioni', 'in', 'Marocco', ',', 'aumenti', 'di', 'prezzo', 'per', 'grano', ',', 'olio', ',', 'zucchero', ',', 'oltre', 'che', 'del', 'petrolio', '.'], ['La', 'sommossa', 'ricorda', 'quella', "dell'", 'assalto', 'al', 'forno', 'delle', 'grucce', ',', 'di', 'manzoniana', 'memoria', '.'], ['È', 'di', 'nuovo', 'la', '"', 'rivolta', 'del', 'pane', '"', ',', 'in', 'tutto', 'il', 'Magreb', '.'], ["C'", 'è', 'chi', 'ricorda', 'che', 'ciclicamente', 'si', 'ripetono', 'le', 'stesse', 'scene', ',', "c'", 'è', 'chi', 'invece', 'vede', "l'", 'inizio', 'di', 'un', 'periodo', 'sempre', 'più', 'negativo', 'sul', 'problema', 'alimentare', 'mondiale', '.'], ['I', 'paesi', 'produttori', 'ed', 'esportatori', 'di', 'grano', ',', 'come', 'la', 'Francia', ',', 'stanno', 'assumendo', 'un', 'atteggiamento', 'sempre', 'più', 'pr

# Metadata Triples Section

In [ ]:
def generate_triples_labelled(dataset_list):

    triples = []

    for i in range(len(dataset_list)):
        for j in range(len(dataset_list[i]['article'])):
            sentence = dataset_list[i]['article'][j]
            tags = dataset_list[i]['tags'][j]

            current_group = []
            current_tags = []

            for idx, word in enumerate(sentence):
                if tags[idx] != 'O':
                    current_group.append(word)
                    current_tags.append(tags[idx])

                    # If the next tag is not an 'I-' tag, we have reached the end of the group
                    if idx + 1 == len(tags) or not tags[idx + 1].startswith('I'):
                        triple_entity = {
                            'subject': ' '.join(current_group),
                            'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
                            'object': current_tags[0].split('-')[1]
                        }
                        triple_article = {
                            'subject': ' '.join(current_group),
                            'predicate': 'http://purl.org/ontology/bibo/citedBy',
                            'object': dataset_list[i]['id']
                        }
                        triples.append(triple_entity)
                        triples.append(triple_article)
                        current_group = []
                        current_tags = []
                else:
                    if current_group:
                        triple = {
                            'subject': ' '.join(current_group),
                            'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
                            'object': current_tags[0].split('-')[1]
                        }
                        triple_article = {
                            'subject': ' '.join(current_group),
                            'predicate': 'http://purl.org/ontology/bibo/citedBy',
                            'object': dataset_list[i]['id']
                        }
                        triples.append(triple_entity)
                        triples.append(triple_article)
                        current_group = []
                        current_tags = []

    return triples

In [ ]:
triples = generate_triples_labelled(dataset_list)

In [ ]:
print(type(triples[1]))
print(triples[2])
print(triples[3])

<class 'dict'>
{'subject': 'Merolone', 'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'object': 'PER'}
{'subject': 'Merolone', 'predicate': 'http://purl.org/ontology/bibo/citedBy', 'object': 0}


In [ ]:
import json

file_path = '/content/drive/MyDrive/Colab Notebooks/TESI/github/triples_withlabel.json'

# Write to the JSON file in Google Drive
with open(file_path, 'w') as file:
    json.dump(triples, file, indent=4)

Generate triples for the two articles only


In [ ]:
def generate_example_triples_labelled(dataset_example):

    triples = []

    for j in range(len(dataset_example['article'])):
          sentence = dataset_example['article'][j]
          tags = dataset_example['tags'][j]

          current_group = []
          current_tags = []

          for idx, word in enumerate(sentence):
              if tags[idx] != 'O':
                  current_group.append(word)
                  current_tags.append(tags[idx])

                  # If the next tag is not an 'I-' tag, we have reached the end of the group
                  if idx + 1 == len(tags) or not tags[idx + 1].startswith('I'):
                      triple_entity = {
                            'subject': ' '.join(current_group),
                            'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
                            'object': current_tags[0].split('-')[1]
                        }
                      triple_article = {
                            'subject': ' '.join(current_group),
                            'predicate': 'http://purl.org/ontology/bibo/citedBy',
                            'object': dataset_example['id']
                        }
                      triples.append(triple_entity)
                      triples.append(triple_article)
                      current_group = []
                      current_tags = []
              else:
                  if current_group:
                      triple = {
                            'subject': ' '.join(current_group),
                            'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
                            'object': current_tags[0].split('-')[1]
                        }
                      triple_article = {
                            'subject': ' '.join(current_group),
                            'predicate': 'http://purl.org/ontology/bibo/citedBy',
                            'object': dataset_example['id']
                        }
                      triples.append(triple_entity)
                      triples.append(triple_article)
                      current_group = []
                      current_tags = []
    return triples

In [ ]:
first_article_triples = generate_example_triples_labelled(dataset_list[1])
second_article_triples = generate_example_triples_labelled(dataset_list[5])

print(len(first_article_triples))
print(len(second_article_triples))

20
28


In [ ]:
import json

example_triples = generate_example_triples_labelled(dataset_list[1]) + generate_example_triples_labelled(dataset_list[5])
print(len(example_triples))

48


In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/TESI/github/example_triples_withlabel.json'

# Write to the JSON file in Google Drive
with open(file_path, 'w') as file:
    json.dump(example_triples, file, indent=4)